In [22]:
from dotenv import load_dotenv
import os

import pandas as pd
import requests


load_dotenv()

True

# Константы

In [23]:
TOKEN_MOSCOW = os.getenv('TOKEN_MOSCOW')

In [24]:
URL_BASE = 'apidata.mos.ru'

In [25]:
params = {
    'api_key': TOKEN_MOSCOW
}

## Получить текущую версию API

In [26]:
url_version = f'https://{URL_BASE}/version'

In [27]:
response = requests.get(url_version, params=params)

In [28]:
print('Версия API: ', response.json().get('Version'))

Версия API:  1


## Получить список наборов данных

In [29]:
url_datasets = f'https://{URL_BASE}/v1/datasets'

In [30]:
params_datasets = params.copy()
params_datasets['$inlinecount'] = 'allpages'
params_datasets['$top'] = 20


In [31]:
response = requests.get(url_datasets, params=params_datasets)

In [32]:
df_datasets = pd.DataFrame(response.json().get('Items'))

In [33]:
df_datasets.tail()

,CategoryId,PublishDate,Keywords,IdentificationNumber,ContainsGeodata,IsSeasonal,IsNew,VersionNumber,Caption,ContainsAccEnvData,IsForeign,IsArchive,SefUrl,Id,ReleaseNumber,DepartmentId,Season,FullDescription,LastUpdateDate
15,104,2013-10-07,"бесплатное детское питание, грудничок, детский...",7707089084-Dairyspecialties,True,False,False,2,Молочные кухни,True,False,False,None,512,390,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2025-01-06
16,104,2013-10-07,"акушеры, беременность, здоровье, здравоохранен...",7707089084-MaternityHomes,True,False,False,2,Родильные дома,True,False,False,None,514,361,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2024-12-27
17,104,2013-10-07,"донор, донорство, здоровье, здравоохранение, к...",7707089084-donation,True,False,False,2,Станции переливания крови,True,False,False,None,515,320,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2024-12-27
18,104,2013-10-31,"бригада скорой помощи, вызвать врача, вызов ск...",7707089084-medicalemergency,True,False,False,2,Скорая и неотложная медицинская помощь,True,False,False,None,516,169,1002,0,"<p style=""margin-left:0cm; margin-right:0cm; t...",2025-01-03
19,104,2013-10-07,"больница, больница клиническая, больницы взрос...",7707089084-clinics,True,False,False,2,Больницы взрослые,True,False,False,None,517,522,1002,0,"<p><span style=""font-family:tahoma,geneva,sans...",2024-12-27


## Получить содержимое наборов данных

In [52]:
ID_DATASET = 658

url_datasets_rows = f'https://{URL_BASE}/v1/datasets/{ID_DATASET}/rows'

In [53]:
params_datasets_rows = params.copy()
params_datasets_rows['$orderby'] = 'global_id'
params_datasets_rows['$top'] = 1000

In [54]:
response = requests.get(url_datasets_rows, params=params_datasets_rows)

Обработка выгруженного диапазона в data_tickets

In [55]:
data_tickets = []

In [56]:
for elem in response.json():
    cell = elem.get('Cells')
    data_tickets.append({
        'global_id': elem.get('global_id'),
        'id_cell' : cell.get('ID'),
        'name_of_tariff': cell.get('NameOfTariff'),
        'type_of_transport': cell.get('TypeOfTransport'),
        'ticket_cost': cell.get('TicketCost'),
        'ticket_validity': cell.get('TicketValidity'),
    })

In [60]:
df_tickets = pd.DataFrame(data_tickets)

In [64]:
df_columns = [
    "global_id", "id строки", 'Наименование тарифа', 'Типа транспорта','Стоимость билетов', 'Начало активации'
]

df_tickets.columns = df_columns

In [66]:
df_tickets.sort_values(by='Стоимость билетов', ascending=False)[:10]

,global_id,id строки,Наименование тарифа,Типа транспорта,Стоимость билетов,Начало активации
20,1029846586,8100,«Единый» на 365 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",25700,"365 дней с момент первого прохода, но не более..."
5,19368725,1300,«Единый» на 365 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",20500,"365 дней с момент первого прохода, но не более..."
26,2388962130,9100,«ТАТ» на 365 дней,"Автобус, электробус, трамвай",14750,"365 дней с момент первого прохода, но не более..."
19,1029846584,8000,«Единый» на 90 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",8850,"90 дней с момент первого прохода, но не более ..."
4,19368723,1200,«Единый» на 90 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",6950,"90 дней с момент первого прохода, но не более ..."
25,2388962108,9000,«ТАТ» на 90 дней,"Автобус, электробус, трамвай",4800,"90 дней с момент первого прохода, но не более ..."
33,2608943181,9504,Единый льготный проездной билет для обучающихс...,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3610,"90 дней с момент первого прохода, но не более ..."
18,1029846583,7800,«Единый» на 30 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3450,"30 дней с момент первого прохода, но не более ..."
1,19368716,700,«Единый» не более 60 поездок,"Метрополитен, монорельс, МЦК, автобус, трамвай...",3420,"45 дней, включая день продажи"
3,19368722,1100,«Единый» на 30 дней,"Метрополитен, монорельс, МЦК, автобус, трамвай...",2870,"30 дней с момент первого прохода, но не более ..."
